In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import category_encoders as ce
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN


/Users/varunchowdary/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/varunchowdary/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('/Users/varunchowdary/Desktop/Credit_Card_Detection/Credit_Fraud_Detection/DataSets/fraud_test.csv')

In [3]:
y = data["is_fraud"]
X = data.drop(['is_fraud'],axis=1)
     

#Modify columns
X_used = X.drop(['Unnamed: 0','street','city','state','cc_num', 'unix_time', 'trans_num'], axis=1)
X_used[['date', 'time']] = X_used['trans_date_trans_time'].str.split(' ', expand=True)
X_used[['day', 'month', 'year']] = X_used['date'].str.split('/', expand=True)
X_used['time'] = X_used['time'].apply(lambda x: x.split(':')[0])
X_used['dob'] = X_used['dob'].apply(lambda x: x.split('/')[-1])
X_used["gender"] = X_used["gender"].replace({'M': 0, 'F': 1})
X_used = X_used.drop(['trans_date_trans_time','date'], axis=1)

/var/folders/v5/dzlv9h1j1m35th_vy031s8d40000gn/T/ipykernel_12611/1512199050.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_used["gender"] = X_used["gender"].replace({'M': 0, 'F': 1})


In [4]:
used_data = X_used.copy()
used_data['is_fraud'] = y

features = used_data.select_dtypes(np.object_).columns
cat_features = []
for feature in features:
    used_data[feature] = used_data[feature].astype('category')
    cat_features.append(feature)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(used_data.drop('is_fraud', axis = 1),
                                                    used_data['is_fraud'],
                                                    stratify = used_data['is_fraud'],
                                                    test_size = 0.25)

train_data = X_train.copy()
train_data['is_fraud'] = y_train

In [8]:
cols_encoder = cat_features

encoder = ce.BinaryEncoder(cols=cols_encoder)

tra_encoder = encoder.fit_transform(X_train[cols_encoder])
test_encode = encoder.transform(X_test[cols_encoder])

X_train = X_train.drop(columns=cols_encoder).join(tra_encoder)

X_test = X_test.drop(columns=cols_encoder).join(test_encode)

In [ ]:
#Doing oversampling to help with the imbalanced
ovs = RandomOverSampler(sampling_strategy = "auto", random_state = 11)
x_rovs, y_rovs = ovs.fit_resample(X_train, y_train)

Using code from kaggle on using nueral networks to help with classifcation: https://www.kaggle.com/code/guptanuj890/creditcardfraudtransaction-with-98-f-score